*Notebook for comparing RAG based on vector indexes vs Knowledge graphs*

Tools and services used:
- Databases: Neo4j Desktop(local neo4j server) to store vectors for vector RAG and  Neo4j Aura DB instance store knowledge graphs.
- LLM-graph-builder : https://llm-graph-builder.neo4jlabs.com/, tool used to convert pdf documents to knowledge graphs.
- LLMs: Gemini 1.5 for knowledge graph generation, Gemini-pro for cypher query mapping and llama3.2:8b for RAG.
- Ollama: Serving Llama3.2, for embedding generations and prompting in RAG.
- RAG orchestration: Langchain components

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings.ollama import OllamaEmbeddings
import os
from dotenv import load_dotenv
from langchain.graphs import Neo4jGraph

*Datasets:*

We will be using wiki documents on Formula1 championship for performing RAG, and comparing by performing set of queries on the documents with RAGs. The documents are stored in the \data folder.

In [ ]:
#drivers to connect to neo4j-graph running locally. We will be using this for vector based RAG.
url = "bolt://localhost:7689"
username ="neo4j"
password = "password"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

**Vector RAG**

We split documents into chunks and embed them using llama3.2 Ollama embeddings, to be used for vector RAG.

In [ ]:
# Define the folder containing PDF files
folder_path = "./data"

# Initialize an embedding model
embeddings = OllamaEmbeddings(model="llama3.2")

# Load all PDF files from the folder
pdf_documents = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(folder_path, file_name)
        loader = PyPDFLoader(file_path)
        pdf_documents.extend(loader.load())

# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pdf_documents)

In [ ]:
# Connect to Neo4j and save embeddings
db = Neo4jVector.from_documents(
    docs,
    embedding=embeddings,
    graph = graph
)

print("PDF documents have been successfully processed and saved to Neo4j!")

In [ ]:
#Creates a retriever to fetch the top 5 most similar vectors from the database using similarity search.
vector_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms.ollama import Ollama

llm = Ollama(model="llama3.2")

#Sets up a Retrieval-Augmented QA pipeline where the LLM answers queries using relevant information retrieved from the vector database.
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_retriever,
    return_source_documents=False
)

![image.png](attachment:image.png)

We’ve implemented the flow using LangChain components, where we’re performing RAG with vector-based search. Here, we’re using Llama3.2:8b as the LLM. It takes the input query, retrieves context by finding similar vector embeddings, and then combines the context with the query to generate a response. Let's try some queries.

In [ ]:
rag_chain.invoke("who is the best?")

In [ ]:
rag_chain.invoke("Who has most wins in 2024?")


In [ ]:
rag_chain.invoke("Who will win in 2024?")


In [ ]:
rag_chain.invoke("Who came second in 2023?")


In [ ]:
rag_chain.invoke("Who is the best driver and best teams across the championships 2022-24")


**Graph RAG**

*Knowledge Graph generation and setup:*

To ensure a fair comparison for RAG, I decided to use the same source of data for both setups. Initially, I planned to create Knowledge Graphs from the documents using LLMGraphTransformer, but I faced issues getting it to work with the Gemini LLM, the only free-supported option. As a workaround, I opted to use Neo4j's *llm-graph-builder* tool, available at *https://llm-graph-builder.neo4jlabs.com/*, which worked effectively for my needs.

The Neo4j LLM Knowledge Graph Builder is an online tool that transforms unstructured text into a knowledge graph, offering a seamless and intuitive text-to-graph experience.

It leverages multiple ML models (LLMs), including OpenAI, Gemini, Llama3, Diffbot, Claude, and Qwen, to process various sources like PDFs, documents, images, web pages, and YouTube video transcripts. The tool extracts relevant information to create two main graph structures:

A lexical graph of documents and chunks (with embeddings)
An entity graph with nodes representing entities and their relationships
Both of these graphs are stored in your Neo4j database.

You can customize the extraction schema and perform clean-up operations after the extraction. Once the data is organized, you can utilize various RAG approaches (GraphRAG, Vector, Text2Cypher) to query your data, gaining insights based on the extracted information and observing how it contributes to constructing accurate answers.

### Steps to Generate a Knowledge Graph using the llm-graph-builder Tool  

1. **Upload Documents**  
   - Upload the documents containing the text data to the **llm-graph-builder** tool.

2. **Select the LLM Model**  
   - Choose the desired **LLM model** to perform entity extraction and graph creation.

3. **Configure Extraction Parameters**  
   - Set up any necessary **extraction parameters** or options to customize the entity and relationship extraction process.

4. **Click "Generate Graph"**  
   - Click the **"Generate graph"** button to start the graph creation process.

5. **LLM Processes the Data**  
   - The selected LLM analyzes the text, **extracting entities** (e.g., names, locations, concepts) and identifying **relationships** among them (e.g., "Person A works at Company B").

6. **Construct the Knowledge Graph**  
   - The tool organizes the extracted entities and relationships into a **graph structure**, creating nodes (entities) and edges (relationships).

7. **Visualize the Graph**  
   - Finally, the tool displays the Knowledge Graph in an **interactive interface**, showing a clear, visual layout of interconnected entities and their relationships.


![image.png](attachment:image.png)

![image.png](attachment:image.png)

![image.png](attachment:image.png)

It resulted in 5,364 nodes and 37,055 relationships, effectively capturing a vast amount of entities and their connections within the knowledge graph.

In [ ]:
#using neo4j-aura instance, since we get graph from the tool, and it create large number of nodes.
url = "neo4j+s://66c55e03.databases.neo4j.io:7687"
username ="neo4j"
password = "QQOiQX0Hr983z3IuQtdL7QETKdyWb41ZIqMp2j0FF_Q"

#the documents are converted to knowledge graphs and added to this server.
graph2 = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

![image.png](attachment:image.png)

This setup is for GraphRAG, also known as Hybrid RAG, which combines context from both unstructured text and knowledge graphs, effectively integrating information from multiple sources to enhance data retrieval and reasoning.

*Creating retriever for unstructured data.*

This code sets up a hybrid search index in Neo4j by combining graph traversal and vector similarity search. It uses the Llama3.2 model from Ollama to generate text embeddings, which are stored in the graph's nodes labeled as 'Chunk' under the embedding property. The Neo4jVector.from_existing_graph function indexes an existing graph, enabling efficient hybrid search queries that combine relationships in the graph structure with semantic similarity search on text embeddings, facilitating better retrieval and reasoning over the data.

In [ ]:
#creating vector indices to be used by RAG . Creates embeddings using Ollama llama llm, and stores them in neo4j
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.ollama import OllamaEmbeddings


vector_index = Neo4jVector.from_existing_graph(
    OllamaEmbeddings( model="llama3.2",),
    graph = graph2,
    search_type="hybrid",
    node_label='Chunk',
    text_node_properties=["text"],
    embedding_node_property='embedding',
)

In [ ]:
#returning similar vector (cosine similarity)
response = vector_index.similarity_search(
    "who is max?"
)
print(response)

In [ ]:
vector_keyword_retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
#return llm's response, on query+rag_context node vector input to LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama
import ollama_function


vector_keyword_chain= RetrievalQA.from_chain_type(
    llm=ollama_function.OllamaFunctions(model="llama3.2"),
    chain_type="stuff",
    retriever=vector_keyword_retriever,
    return_source_documents=False
)

Now, we have half the rag, which get unstructured context using both vector and text indexes. Let's try running some queries with it alone.

In [ ]:
vector_keyword_chain.invoke("who is the best driver?")

In [ ]:
vector_keyword_chain.invoke("Who has most wins in 2024?")


In [ ]:
vector_keyword_chain.invoke("Who will win in 2024?")

In [ ]:
vector_keyword_chain.invoke("Who came second in 2023?")

*Creating retriever for structured data.*

Configuring a graph retrieval setup is more complex but provides greater flexibility. In this example, we use a full-text index to find relevant nodes and then retrieve their direct neighborhood. The graph retriever begins by identifying pertinent entities in the input, such as people, organizations, and locations. To accomplish this, we employ LCEL along with the newly introduced with_structured_output method.

![image.png](attachment:image.png)

In [ ]:

from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from typing import Tuple, List, Optional
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")


llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True,GOOGLE_API_KEY=GOOGLE_API_KEY)
graph2.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "Who is max verstappen?"}).names

We can see that it is able to get the entities from the query.

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
graph2.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text
    search. It processes the input string by splitting it into words and
    appending a similarity threshold (~2 changed characters) to each
    word, then combines them using the AND operator. Useful for mapping
    entities from user questions to database values, and allows for some
    misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [ ]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    print(entities.names)
    for entity in entities.names:
        response = graph2.query(
            """CALL db.index.fulltext.queryNodes('entities', $query,
            {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS
              output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS
              output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        # for el in response:
        #     print(el['output'])
        result += "\n".join([el['output'] for el in response if el['output'] is not None])
    return result

In [ ]:

print(structured_retriever("Who is Max Verstappen?"))

We can see the graph retreiver is able to retrieve the related nodes from the graph. Now we can combine these outputs with vector retriever output, as part of a rag chain.

In [ ]:
repsonse = vector_index.similarity_search("who is verstappen?")
print("received data .....")

for el in response:
    print(el.page_content)

In [ ]:
#This retriever combines both vector and graph retrievers for same question.
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(final_data)
    return final_data

In [ ]:
# llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
llm = Ollama(model="llama3.2")

Here’s the RAG chain setup: it takes an input query, retrieves a combined context from both vector embeddings and graph data, and then passes this along with the query to the LLM. The LLM generates an output, which is subsequently refined and printed.

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.output_parsers.string import StrOutputParser

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = chain.invoke(input="Who is max verstappen?")
print("Final Answer")
print(response)

The output explains the hybrid retrieval process for RAG by describing how the system combines vector and graph search approaches. It first retrieves context by searching through the vector index using embedding vectors and keywords. Then, it searches the knowledge graph to find related entities and their connections. These two contexts—vector-based and graph-based—are combined and fed into the LLM. The LLM then processes the combined information to generate the final, accurate answer. This hybrid approach leverages both unstructured data (through embeddings and keyword search) and structured data (through graph connections) to enhance the retrieval process in RAG applications.

In [ ]:
response = chain.invoke(input="Who will win 2024 f1 championship?")
print("Final Answer")
print(response)

In [ ]:
response = chain.invoke(input="Who won 2023 f1 championship?")
print("Final Answer")
print(response)

In [ ]:
response = chain.invoke(input="Who came second in 2023 f1 championship?")
print("Final Answer")
print(response)

In [ ]:
response = chain.invoke(input="Which team won in 2023 f1 championship")
print("Final Answer")
print(response)